In [1]:
import os, sys

sys.path.append(os.path.abspath(os.path.join('..')))

import matplotlib.pyplot as plt

from utils.visualization import data_count_plot
from utils.duplicates import find_near_duplicates, visualize_duplicates, get_duplicates_to_delete

from utils import data

### Loading Dataframe

In [2]:
base_path = '../data/raw/Furniture_Data'
df = data.load(base_path)

Loading tables...
Loading tables/Eclectic...
Loading tables/Midcentury...
Loading tables/Craftsman...
Loading tables/Victorian...
Loading tables/Transitional...
Loading tables/Mediterranean...
Loading tables/Traditional...
Loading tables/Industrial...
Loading tables/Farmhouse...
Loading tables/Tropical...
Loading tables/Southwestern...
Loading tables/Beach...
Loading tables/Modern...
Loading tables/Contemporary...
Loading tables/Rustic...
Loading tables/Scandinavian...
Loading tables/Asian...
Loading sofas...
Loading sofas/Eclectic...
Loading sofas/Midcentury...
Loading sofas/Craftsman...
Loading sofas/Victorian...
Loading sofas/Transitional...
Loading sofas/Mediterranean...
Loading sofas/Traditional...
Loading sofas/Industrial...
Loading sofas/Farmhouse...
Loading sofas/Tropical...
Loading sofas/Southwestern...
Loading sofas/Beach...
Loading sofas/Modern...
Loading sofas/Contemporary...
Loading sofas/Rustic...
Loading sofas/Scandinavian...
Loading sofas/Asian...
Loading lamps...
Loadi

In [ ]:
df.describe()

In [ ]:
df.head()

### Histogram Plot

In [ ]:
def plot_histogram(df, column_name, bins=20, title=None):
    plt.figure(figsize=(10, 5))
    plt.hist(df[column_name], bins=bins, color='skyblue', edgecolor='black')
    plt.title(title or f'Distribution of {column_name}')
    plt.xlabel(column_name)
    plt.ylabel('Frequency')
    plt.show()

In [ ]:
# Plot histograms for numerical columns
plot_histogram(df, 'Width', title='Histogram of Image Widths')
plot_histogram(df, 'Height', title='Histogram of Image Heights')
plot_histogram(df, 'Ratio', title='Histogram of Image Ratios')

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(15, 10))

data_count_plot(df, 'Type', ax=ax[0][0], title='File Types', annotate=True, palette='Set2')
data_count_plot(df, 'Mode', ax=ax[0][2], title='Image Modes', annotate=True, palette='Accent')
data_count_plot(df, 'Class', ax=ax[1][1], horizontal=True, title='Classes', annotate=True, palette='Set1')
data_count_plot(df, 'Style', ax=ax[1][2], horizontal=True, title='Styles', annotate=True, palette='Set1')

fig.suptitle('Raw Dataset Statistics', fontsize=20, fontweight='bold', y=1.0)
fig.tight_layout()

---
## Duplicate handling

### Find all duplicates or similar images

In [ ]:
duplicates = find_near_duplicates(df, threshold=1)
duplicates

Processing images:  26%|██▌       | 23023/90084 [06:41<36:25, 30.68image/s, Duplicates=1298, Current=../data/raw/Furniture_Data/lamps/Craftsman/26698craftsman-table-lamps.jpg]                              

### Visualize duplicates images to test

In [ ]:
visualize_duplicates(duplicates, num_groups=5, start_group=1)

### Get duplicates to be deleted 

- This only includes "Duplicate" images (and not "Similar" images). 
- 1 "Duplicate" image from a Duplicate Group will be kept (not included in the output of duplicate_to_delete()).

In [ ]:
to_be_deleted = get_duplicates_to_delete(duplicates)
to_be_deleted

## Delete duplicates

In [8]:
# Uncomment below
# df = df[~df['Path'].isin(images_to_delete['Path'])]
